# MAC0460: Exercício Programa 2

**Aluno**: Dérick William de Moraes Frias

**NUSP**: 11207623

# Abstrato

Este código é o projeto de um classificador binário de 'tipo 1' de pokemons. Este classificador binário decide tipos de pokemons entre 'water type' e 'normal type'.

A classificação de pokemons é feita com base apenas nas características físicas e de saude dos pokemons. Além disto, a classificação dos pokemons será feita com a ajuda do Pikachu, um pokemon de tipo elétrico, sendo assim podemos usar a característica 'against_electric' dos pokemons no nosso classificador. Como não queremos que nosso Pikachu se machuque, descartamos no classificador os campos referentes aos ataques de cada pokemon. Por fim, o tipo secundário dos pokemons é ignorado.

O objetivo deste projeto é estudar o comportamento dos algoritmos SVM, Decision Tree, Random Forest e Linear Regression, com amostras de dados normalizadas e não-normalizadas.

O código é dividido em seções curtas, e deve ser executado sequencialmente.

Antes de executar, importe pokedata.csv.


### Pokedata

Get pokedata.csv at https://www.kaggle.com/datasets/rounakbanik/pokemon

#Montagem do Dataframe

## Imports


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import numpy as np

from sklearn.model_selection import GridSearchCV, KFold
from sklearn import metrics


### Definindo Seed Global


In [2]:
seed = 7542

### Carregando Dados dos Pokemons


In [4]:
df = pd.read_csv("sample_data/pokedata.csv") #Can be downloaded at https://www.kaggle.com/datasets/rounakbanik/pokemon

#Infos gerais do Dataframe

In [5]:
df.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   abilities          801 non-null    object 
 1   against_bug        801 non-null    float64
 2   against_dark       801 non-null    float64
 3   against_dragon     801 non-null    float64
 4   against_electric   801 non-null    float64
 5   against_fairy      801 non-null    float64
 6   against_fight      801 non-null    float64
 7   against_fire       801 non-null    float64
 8   against_flying     801 non-null    float64
 9   against_ghost      801 non-null    float64
 10  against_grass      801 non-null    float64
 11  against_ground     801 non-null    float64
 12  against_ice        801 non-null    float64
 13  against_normal     801 non-null    float64
 14  against_poison     801 non-null    float64
 15  against_psychic    801 non-null    float64
 16  against_rock       801 non

# Preparação dos Dados no Dataframe

Aqui vamos preparar os dados para o uso em cada modelo que usaremos

### Removendo colunas não utilizáveis


In [7]:
dropped=['abilities', 'against_bug', 'against_dark', 'against_dragon',
        'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'classfication',
       'japanese_name', 'name', 'pokedex_number', 'type2','generation', 'percentage_male', 'capture_rate']

df=df.drop(dropped, axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   against_electric   801 non-null    float64
 1   attack             801 non-null    int64  
 2   base_egg_steps     801 non-null    int64  
 3   base_happiness     801 non-null    int64  
 4   base_total         801 non-null    int64  
 5   defense            801 non-null    int64  
 6   experience_growth  801 non-null    int64  
 7   height_m           781 non-null    float64
 8   hp                 801 non-null    int64  
 9   sp_attack          801 non-null    int64  
 10  sp_defense         801 non-null    int64  
 11  speed              801 non-null    int64  
 12  type1              801 non-null    object 
 13  weight_kg          781 non-null    float64
 14  is_legendary       801 non-null    int64  
dtypes: float64(3), int64(11), object(1)
memory usage: 94.0+ KB


In [ ]:
df.head()

### Seleciona pokemons 'Water' e 'Normal' e redefine tipos


In [ ]:
# seleciona pokemons 'water' e 'normal'

#df = df[pd.to_numeric(df['capture_rate'], errors = 'coerce').notnull()]
#df['capture_rate'] = df['capture_rate'].astype('int')

df = df[df['type1'].isin(['water', 'normal'])]
df['type1'] = df['type1'].replace({'water':1, 'normal':0})

### Remove NaNs




In [ ]:
df = df.dropna()

### Infos gerais do main dataframe




In [ ]:
df.info()



In [ ]:
df

In [ ]:
df['type1'].value_counts()
#dfnormal = df[df['type1']==0]
#dfwater = df[df['type1']==1]

## Separando features em treino e teste




In [ ]:
feature_df = df[['against_electric', 'attack', 'base_egg_steps', 'base_happiness', 'base_total',
                 'defense', 'experience_growth', 'height_m', 'hp',
                 'sp_attack', 'sp_defense', 'speed',
                 'weight_kg', 'is_legendary']]

X = np.asarray(feature_df)
Y = np.asarray(df['type1'])



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)




# Montagem dos modelos & chamadas dos modelos




## SVM's

####Desliga Warnings
Esta execução dispara warning:

 *warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.*

Inúmeras vezes. Tanto que compromete a saida grafica do processamento do algoritmo. Antes de executar as células, desligue os warnings.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

####Religa Warnings

In [ ]:
warnings.filterwarnings("default")

####Implementação

In [ ]:
from sklearn import svm

def supvecmachine(X, Y, Xt, Yt):
  svm_param_grid = {
     'C': [0.001, 0.1, 1, 2, 10],
     'gamma': [0.1, 0.5, 1, 2, 'scale', 'auto'],
     'max_iter': [10, 20, 30, 100, 200, 1000],
     'kernel': ['linear', 'rbf']
  }


  svm_classifier = svm.SVC(random_state=seed)

  svm_kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

  grid_search = GridSearchCV(svm_classifier, svm_param_grid, cv=svm_kfold)
  grid_search.fit(X, Y)

  print("Parametros escolhidos: ", grid_search.best_params_)

  svm_best = grid_search.best_estimator_
  svm_best.fit(X, Y)

  svm_Y_predict = svm_best.predict(Xt)
  print("Acuracia:", metrics.accuracy_score(Yt, svm_Y_predict))


####Call SVM

In [ ]:
supvecmachine(X_train, Y_train, X_test, Y_test)

Parametros escolhidos:  {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': 30}
Acuracia: 0.5581395348837209


## Decision Trees

####Implementação

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def decisiontree(X, Y, Xt, Yt):
  dectree_param_grid = {
      'max_depth': [10, 25, 50, 100, 250, 400],
      'min_samples_split': [2, 5, 12, 30, 50, 100],
      'max_depth': [2, 5, 10],
      'splitter': ['best', 'random'],
      'criterion': ['gini', 'entrophy']
  }

  dectree_classifier = DecisionTreeClassifier()

  dectree_kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

  grid_search = GridSearchCV(dectree_classifier, dectree_param_grid, cv=dectree_kfold)

  grid_search.fit(X, Y)

  print("Parametros escolhidos: ", grid_search.best_params_)

  dectree_best = grid_search.best_estimator_
  dectree_best.fit(X, Y)

  dectree_Y_predict = dectree_best.predict(Xt)
  print("Acuracia:", metrics.accuracy_score(Yt, dectree_Y_predict))
  print("Feature mais importante:", np.max(dectree_best.feature_importances_))
  print(dectree_best.feature_importances_)

####Call Decision Tree

In [ ]:
decisiontree(X_train, Y_train, X_test, Y_test)

Parametros escolhidos:  {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 12, 'splitter': 'best'}
Acuracia: 0.8604651162790697
Feature mais importante: 0.4985891654289236
[0.49858917 0.06912187 0.11416142 0.         0.         0.08794144
 0.07317758 0.         0.         0.04741492 0.         0.06325019
 0.04634342 0.        ]


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 889, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _vali

##Random Forests

####Implementação

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Esta função leva alguns minutos para executar!!!
def randomforest(X, Y, Xt, Yt):
  randfor_param = {
      'criterion':['gini', 'entropy'],
      'n_estimators': [50, 100, 200, 300],
      'max_depth': [2, 5, 10],
      'class_weight': ['balanced', 'balanced_subsample'],
      'min_samples_split': [2,  5, 12, 30, 50]
  }

  randfor_classifier = RandomForestClassifier(random_state=seed)

  randfor_kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

  randfor_grid = GridSearchCV(randfor_classifier, randfor_param, cv=randfor_kfold)

  randfor_grid.fit(X, Y)

  print("Parametros escolhidos: ", randfor_grid.best_params_)

  randfor_best = randfor_grid.best_estimator_

  randfor_best.fit(X, Y)

  randfor_Y_pred = randfor_best.predict(Xt)
  print("Acuracia:", metrics.accuracy_score(Yt, randfor_Y_pred))
  print("Feature mais importante:", np.max(randfor_best.feature_importances_))
  print(randfor_best.feature_importances_)

#pd.DataFrame(y_pred_rf).to_csv("y-pred-rf.csv", index=False)
#Conclusão em 18 minutos

####Call Random Forest

In [ ]:
randomforest(X_train, Y_train, X_test, Y_test)

Parametros escolhidos:  {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 300}
Acuracia: 0.8372093023255814
Feature mais importante: 0.24253306122535154
[0.24253306 0.06866694 0.06084067 0.01529146 0.07044988 0.09215065
 0.03735747 0.0292466  0.05937661 0.12809994 0.06693659 0.06636801
 0.06200137 0.00068077]


## Regressão Linear

In [ ]:
from sklearn.linear_model import LogisticRegression

def regressaolinear(X, Y, Xt, Yt):
  reglin_param = {
      'C': [0.01, 0.1, 1, 10],
      'penalty': ['l1', 'l2'],
      'solver': ['liblinear', 'lbfgs', 'sag', 'newton-cg'],
      'max_iter': [10, 20, 50, 100, 150]
  }

  reglin_classifier = LogisticRegression()

  reglin_kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

  reglin_grid = GridSearchCV(reglin_classifier, reglin_param, cv=reglin_kfold)

  reglin_grid.fit(X, Y)

  print("Parametros escolhidos: ", reglin_grid.best_params_)

  reglin_best = reglin_grid.best_estimator_
  reglin_best.fit(X, Y)

  reglin_Y_pred = reglin_grid.predict(Xt)
  print("Acuracia:", metrics.accuracy_score(Yt, reglin_Y_pred))
  print(np.max(reglin_best.coef_))
  print(reglin_best.coef_)
  #reglin_grid.bash



In [ ]:
regressaolinear(X_train, Y_train, X_test, Y_test)

Parametros escolhidos:  {'C': 0.01, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
Acuracia: 0.627906976744186
0.10653301593410562
[[ 1.06533016e-01 -3.90209291e-02 -4.97328478e-05  1.11607833e-02
   1.47910558e-03  4.65734664e-02  2.65732296e-06  9.91538591e-03
  -2.83940987e-02  4.91060007e-02 -9.86555639e-03 -1.69197774e-02
   1.36659413e-03  2.78938185e-03]]


# Normalização dos dados de treino e validação & novas execuções

## Normalização

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

## Nova execução dos modelos com dados normalizados

### SVM's

In [ ]:
supvecmachine(X_train_norm, Y_train, X_test_norm, Y_test)

Parametros escolhidos:  {'C': 2, 'gamma': 0.5, 'kernel': 'rbf', 'max_iter': 200}
Acuracia (sem normalizacao): 0.7906976744186046


### Decision Trees

In [ ]:
decisiontree(X_train_norm, Y_train, X_test_norm, Y_test)

Parametros escolhidos:  {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 12, 'splitter': 'best'}
Acuracia (sem normalizacao): 0.8372093023255814


### Random Forests

In [ ]:
randomforest(X_train_norm, Y_train, X_test_norm, Y_test)

Parametros escolhidos:  {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 300}
Acuracia (sem normalizacao): 0.8604651162790697


### Regressão Linear

In [ ]:
regressaolinear(X_train_norm, Y_train, X_test_norm, Y_test)

Parametros escolhidos:  {'C': 1, 'max_iter': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Acuracia (sem normalizacao): 0.7906976744186046
